# Import Libs

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers 
import tensorflow_addons as tfa

from keras.regularizers import l2
from keras.utils import to_categorical
from keras.models import load_model

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Load Name of files and your labels

In [ ]:
X_train = np.load('/kaggle/input/birdclef-2020/X_train.npy')
y_train = np.load('/kaggle/input/birdclef-2020/y_train.npy')

X_val = np.load('/kaggle/input/birdclef-2020/X_val.npy')
y_val = np.load('/kaggle/input/birdclef-2020/y_val.npy')

In [ ]:
X_train.shape

# Organize data to all batchs have positives and negatives anchors

In [ ]:
# Sort by Label
y_train,X_train = (list(t) for t in zip(*sorted(zip(y_train, X_train))))
y_val,X_val = (list(t) for t in zip(*sorted(zip(y_val, X_val))))

In [ ]:
#Organize to all batchs have positives and negatives anchors
tuple_X = []
tuple_y = []

for i in range(0,len(y_train),2):
    if(i == len(y_train)-1):
        tuple_X.append((X_train[i],X_train[i]))
        tuple_y.append((y_train[i],y_train[i]))
    else:    
        tuple_X.append((X_train[i],X_train[i+1]))
        tuple_y.append((y_train[i],y_train[i+1]))

tuple_X_shuffled, tuple_y_shuffled = shuffle(tuple_X, tuple_y)

X_shuffled = []
y_shuffled = []
for i in range(0,len(tuple_y_shuffled)):
    X_shuffled.append(tuple_X_shuffled[i][0])
    X_shuffled.append(tuple_X_shuffled[i][1])
    y_shuffled.append(tuple_y_shuffled[i][0])
    y_shuffled.append(tuple_y_shuffled[i][1])

X_train = np.array(X_shuffled)
y_train = np.array(y_shuffled)

In [ ]:
#Organize to all batchs have positives and negatives anchors

tuple_X = []
tuple_y = []

for i in range(0,len(y_val),2):
    if(i == len(y_val)-1):
        tuple_X.append((X_val[i],X_val[i]))
        tuple_y.append((y_val[i],y_val[i]))
    else:    
        tuple_X.append((X_val[i],X_val[i+1]))
        tuple_y.append((y_val[i],y_val[i+1]))

tuple_X_shuffled, tuple_y_shuffled = shuffle(tuple_X, tuple_y)

X_shuffled = []
y_shuffled = []
for i in range(0,len(tuple_y_shuffled)):
    X_shuffled.append(tuple_X_shuffled[i][0])
    X_shuffled.append(tuple_X_shuffled[i][1])
    y_shuffled.append(tuple_y_shuffled[i][0])
    y_shuffled.append(tuple_y_shuffled[i][1])

X_val = np.array(X_shuffled)
y_val = np.array(y_shuffled)

# Make a DataGenerator to get the numpy arrays

In [ ]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, image_filenames,labels, image_path,to_fit=True, batch_size=64):
        self.labels = labels
        self.image_filenames = image_filenames
        self.image_path = image_path
        self.to_fit = to_fit
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        # Generate data
        X = self._generate_X(index)

        if self.to_fit:
            y = self._generate_y(index)
            return X, y
        else:
            return X

    def _generate_X(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        ret = []

        for file_name in batch_x:
            dict_data = np.load(self.image_path +str(file_name)+".npz")
            data = dict_data['arr_0']
            ret.append(data)

        return np.array(ret)

    def _generate_y(self, idx):
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array(batch_y)

# The Multiscale CNN

In [ ]:
wd = 0.000001

#Template of Multiscale Analysis Module
def MAM(in_mam):
    str1 = layers.Conv2D(64,kernel_size=(1,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(in_mam)
    
    str2 = layers.Conv2D(32,kernel_size=(1,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(in_mam)
    str2 = layers.Conv2D(64,kernel_size=(3,3),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(str2)
    
    str3 = layers.Conv2D(32,kernel_size=(1,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(in_mam)
    str3 = layers.Conv2D(64,kernel_size=(5,5),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(str3)
    
    str4 = layers.Conv2D(32,kernel_size=(1,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(in_mam)
    str4 = layers.Conv2D(64,kernel_size=(7,7),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(str4)
    
    mam = layers.concatenate([str1,str2,str3,str4])
    return mam

#BEGIN OF MULTISCALE CNN
Input = keras.Input(shape=(40, 200, 3))

#CONV1
x = layers.Conv2D(64, kernel_size=(3,3), strides=(1,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(Input)
#MAM1
x = MAM(x)
#CONV2
x = layers.Conv2D(64, kernel_size=3,strides=(2,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
#MAM2
x = MAM(x)
#CONV3
x = layers.Conv2D(64, kernel_size=3,strides=(2,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
#MAM1
x = MAM(x)
#CONV4
x = layers.Conv2D(64, kernel_size=3,strides=(2,1),padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
#MAM1
x = MAM(x)
#CONV5
x = layers.Conv2D(64, kernel_size=3,strides=(5,1), padding="same",activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256,activation='relu',kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
x = layers.Dense(256,activation='relu', kernel_regularizer=l2(wd), bias_regularizer=l2(wd))(x)
emb128 = layers.Dense(256,activation='linear')(x)

model1 = keras.Model(inputs=Input, outputs=emb128, name="Multiscale_CNN")

#To see the summary of model uncoment here
model1.summary()
#To see a plot of model uncoment here
#keras.utils.plot_model(model1, "deep_metric_learning.png", show_shapes=True)

In [ ]:
model1.compile(
        optimizer=keras.optimizers.Adagrad(0.001),
        loss=tfa.losses.TripletSemiHardLoss())

# Train Process

In [ ]:
#CallBack to save the weights of model fo each epoch
checkpoint_filepath = '/kaggle/working/'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'model{epoch:08d}.h5',save_weights_only=True)

In [ ]:
#Create a generator to train and a generator for validation data
image_path = '../input/birdclef-2020/train/train/'
my_train_generator = DataGenerator(X_train, y_train,image_path,batch_size=32)
my_val_generator = DataGenerator(X_val, y_val,image_path,batch_size=32)

In [ ]:
#Train
history = model1.fit(my_train_generator,epochs=30,verbose=1,callbacks=[model_checkpoint_callback])

In [ ]:
# Plot the "Loss"
plt.plot(history.history['loss'][:])
plt.plot(history.history['val_loss'][:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Uncoment here to save the model in the Disk

# serialize model to JSON
model_json = model1.to_json()
with open("MultiScale-Triplet.json", "w") as json_file:
    json_file.write(model_json)
print("Saved model to disk")